# Team IPPS 

The API information can be accessed through this [link](https://data.cms.gov/Medicare-Inpatient/Inpatient-Prospective-Payment-System-IPPS-Provider/97k6-zzx3)

Objectives (This will expand from just data frame formation to include cleanup by Jessie):
1. Execute a successful pull of all of the data using the API provided into JSON
2. Transform the JSON format into a data frame


### Step 1: Installing sodapy, a python package, in jupyter

In [1]:
# Installing the package sodapy, important for retrieval using the API's detailed method
import sys
!{sys.executable} -m pip install sodapy

    100% |████████████████████████████████| 829kB 9.7MB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 11.1MB/s ta 0:00:01
  Running setup.py bdist_wheel for future ... done
  Stored in directory: /Users/mochathereal/Library/Caches/pip/wheels/bf/c9/a3/c538d90ef17cf7823fa51fc701a7a7a910a80f6a405bf15b1a
Successfully built future
distributed 1.21.8 requires msgpack, which is not installed.
  Found existing installation: requests 2.18.4
    Uninstalling requests-2.18.4:
      Successfully uninstalled requests-2.18.4
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### Step 2: Retrieving the dataset

In [2]:
# The API link
link = "https://data.cms.gov/resource/ehrv-m9r6.json"

# The app token
key = "oBbcgRhXZS4dqtTJVyz6zQujv"

# Code snippet for data retrieval using python, as provided by the API information page
import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cms.gov", key)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cms.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("ehrv-m9r6", limit=163065)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

# Checking the head of the data frame
results_df.head(10)

,average_covered_charges,average_medicare_payments,average_medicare_payments_2,drg_definition,hospital_referral_region_description,provider_city,provider_id,provider_name,provider_state,provider_street_address,provider_zip_code,total_discharges
0,32963.07,5777.24,4763.73,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Dothan,DOTHAN,10001,SOUTHEAST ALABAMA MEDICAL CENTER,AL,1108 ROSS CLARK CIRCLE,36301,91
1,15131.85,5787.57,4976.71,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Birmingham,BOAZ,10005,MARSHALL MEDICAL CENTER SOUTH,AL,2505 U S HIGHWAY 431 NORTH,35957,14
2,37560.37,5434.95,4453.79,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Birmingham,FLORENCE,10006,ELIZA COFFEE MEMORIAL HOSPITAL,AL,205 MARENGO STREET,35631,24
3,13998.28,5417.56,4129.16,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Birmingham,BIRMINGHAM,10011,ST VINCENT'S EAST,AL,50 MEDICAL PARK EAST DRIVE,35235,25
4,31633.27,5658.33,4851.44,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Birmingham,ALABASTER,10016,SHELBY BAPTIST MEDICAL CENTER,AL,1000 FIRST STREET NORTH,35007,18
5,16920.79,6653.80,5374.14,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Montgomery,MONTGOMERY,10023,BAPTIST MEDICAL CENTER SOUTH,AL,2105 EAST SOUTH BOULEVARD,36116,67
6,11977.13,5834.74,4761.41,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Birmingham,OPELIKA,10029,EAST ALABAMA MEDICAL CENTER AND SNF,AL,2000 PEPPERELL PARKWAY,36801,51
7,35841.09,8031.12,5858.5,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Birmingham,BIRMINGHAM,10033,UNIVERSITY OF ALABAMA HOSPITAL,AL,619 SOUTH 19TH STREET,35233,32
8,28523.39,6113.38,5228.4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Huntsville,HUNTSVILLE,10039,HUNTSVILLE HOSPITAL,AL,101 SIVLEY RD,35801,135
9,75233.38,5541.05,4386.94,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Birmingham,GADSDEN,10040,GADSDEN REGIONAL MEDICAL CENTER,AL,1007 GOODYEAR AVENUE,35903,34


In [73]:
##state vs provider count
stateGB = results_df.groupby(["provider_state"])
stateGB_providerCt = pd.DataFrame(stateGB["provider_id"].count())
stateGB_providerCt.rename(columns={"provider_id": "Provider Count"}, inplace=True)
stateGB_providerCt
##resultGB.sort_values(["drg_definition", "counts"], ascending = [True, False])
##type(stateGB)

,Provider Count
provider_state,
AK,231
AL,3635
AR,2067
AZ,2851
CA,13064
CO,1890
CT,2011
DC,462
DE,394


In [66]:
## state vs total discharges
results_df["total_discharges"] = results_df["total_discharges"].astype(int)
stateGB = results_df.groupby(["provider_state"])
stateGB_discharges = pd.DataFrame(stateGB["total_discharges"].sum())
stateGB_discharges

,total_discharges
provider_state,
AK,6142
AL,142704
AR,86769
AZ,104604
CA,474979
CO,61320
CT,96258
DC,20307
DE,26753


In [68]:
# state discharges vs DRG count
results_df["total_discharges"] = results_df["total_discharges"].astype(int)
stateGB = results_df.groupby(["provider_state"])
stateGB_dischargesDRG = pd.DataFrame(stateGB["total_discharges"].sum())
stateGB_dischargesDRG["DRG Count"] = stateGB["drg_definition"].nunique()
stateGB_dischargesDRG

,total_discharges,DRG Count
provider_state,,
AK,6142,78
AL,142704,100
AR,86769,100
AZ,104604,100
CA,474979,100
CO,61320,100
CT,96258,100
DC,20307,100
DE,26753,100


In [ ]:
resultGB = results_df.groupby(["provider_state","drg_definition"])#.size().reset_index(name='DRG counts')
drgCtByState = pd.DataFrame(resultGB["provider_id"].count())
drgCtByState.rename(columns = {"provider_id": "Provider Count"}, inplace = True)
drgCtByState

In [81]:
resultGB = results_df.groupby(["provider_state","drg_definition"])#.size().reset_index(name='DRG counts')
drgCtByState = pd.DataFrame(resultGB["provider_id"].count())
drgCtByState.rename(columns = {"provider_id": "Provider Count"}, inplace = True)
drgCtByState
#drgCtByState.sort_values(["drg_definition", "counts"], ascending = [True, False])

Provider Count
provider_state drg_definition                                                    
AK             039 - EXTRACRANIAL PROCEDURES W/O CC/MCC                         1
               057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/O...               1
               064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFAR...               2
               065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFAR...               6
               066 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFAR...               4
               069 - TRANSIENT ISCHEMIA                                         2
               101 - SEIZURES W/O MCC                                           2
               176 - PULMONARY EMBOLISM W/O MCC                                 1
               177 - RESPIRATORY INFECTIONS & INFLAMMATIONS W MCC               2
               178 - RESPIRATORY INFECTIONS & INFLAMMATIONS W CC                3
               189 - PULMONARY EDEMA & RESPIRATORY FAILURE                      3
               190 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W MCC                7
               191 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W CC                 6
               192 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W/O...               6
               193 - SIMPLE PNEUMONIA & PLEURISY W MCC                          6
               194 - SIMPLE PNEUMONIA & PLEURISY W CC                           8
               195 - SIMPLE PNEUMONIA & PLEURISY W/O CC/MCC                     6
               207 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATOR...               2
               208 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATOR...               3
               238 - MAJOR CARDIOVASC PROCEDURES W/O MCC                        2
               243 - PERMANENT CARDIAC PACEMAKER IMPLANT W CC                   1
               244 - PERMANENT CARDIAC PACEMAKER IMPLANT W/O C...               1
               246 - PERC CARDIOVASC PROC W DRUG-ELUTING STENT...               1
               247 - PERC CARDIOVASC PROC W DRUG-ELUTING STENT...               4
               251 - PERC CARDIOVASC PROC W/O CORONARY ARTERY ...               1
               252 - OTHER VASCULAR PROCEDURES W MCC                            1
               253 - OTHER VASCULAR PROCEDURES W CC                             1
               280 - ACUTE MYOCARDIAL INFARCTION, DISCHARGED A...               1
               281 - ACUTE MYOCARDIAL INFARCTION, DISCHARGED A...               1
               282 - ACUTE MYOCARDIAL INFARCTION, DISCHARGED A...               1
...                                                                           ...
WY             419 - LAPAROSCOPIC CHOLECYSTECTOMY W/O C.D.E. W...               1
               439 - DISORDERS OF PANCREAS EXCEPT MALIGNANCY W CC               2
               460 - SPINAL FUSION EXCEPT CERVICAL W/O MCC                      4
               470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT O...              11
               473 - CERVICAL SPINAL FUSION W/O CC/MCC                          3
               480 - HIP & FEMUR PROCEDURES EXCEPT MAJOR JOINT...               1
               481 - HIP & FEMUR PROCEDURES EXCEPT MAJOR JOINT...               3
               482 - HIP & FEMUR PROCEDURES EXCEPT MAJOR JOINT...               2
               491 - BACK & NECK PROC EXC SPINAL FUSION W/O CC...               3
               536 - FRACTURES OF HIP & PELVIS W/O MCC                          1
               552 - MEDICAL BACK PROBLEMS W/O MCC                              4
               563 - FX, SPRN, STRN & DISL EXCEPT FEMUR, HIP, ...               1
               602 - CELLULITIS W MCC                                           1
               603 - CELLULITIS W/O MCC                                         4
               638 - DIABETES W CC                                              2
               640 - MISC DISORDERS OF NUTRITION,METABOLISM,FL...               2
               641 - MISC DISORDERS OF NUTRITION,

In [13]:
resultGB_data = pd.DataFrame(resultGB.count)
resultGB_data

average_covered_charges  \
provider_state drg_definition                                                                
AK             039 - EXTRACRANIAL PROCEDURES W/O CC/MCC                                  1   
               057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/O...                        1   
               064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFAR...                        2   
               065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFAR...                        6   
               066 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFAR...                        4   
               069 - TRANSIENT ISCHEMIA                                                  2   
               101 - SEIZURES W/O MCC                                                    2   
               176 - PULMONARY EMBOLISM W/O MCC                                          1   
               177 - RESPIRATORY INFECTIONS & INFLAMMATIONS W MCC                        2   
               178 - RESPIRATORY INFECTIONS & INFLAMMATIONS W CC                         3   
               189 - PULMONARY EDEMA & RESPIRATORY FAILURE                               3   
               190 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W MCC                         7   
               191 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W CC                          6   
               192 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W/O...                        6   
               193 - SIMPLE PNEUMONIA & PLEURISY W MCC                                   6   
               194 - SIMPLE PNEUMONIA & PLEURISY W CC                                    8   
               195 - SIMPLE PNEUMONIA & PLEURISY W/O CC/MCC                              6   
               207 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATOR...                        2   
               208 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATOR...                        3   
               238 - MAJOR CARDIOVASC PROCEDURES W/O MCC                                 2   
               243 - PERMANENT CARDIAC PACEMAKER IMPLANT W CC                            1   
               244 - PERMANENT CARDIAC PACEMAKER IMPLANT W/O C...                        1   
               246 - PERC CARDIOVASC PROC W DRUG-ELUTING STENT...                        1   
               247 - PERC CARDIOVASC PROC W DRUG-ELUTING STENT...                        4   
               251 - PERC CARDIOVASC PROC W/O CORONARY ARTERY ...                        1   
               252 - OTHER VASCULAR PROCEDURES W MCC                                     1   
               253 - OTHER VASCULAR PROCEDURES W CC                                      1   
               280 - ACUTE MYOCARDIAL INFARCTION, DISCHARGED A...                        1   
               281 - ACUTE MYOCARDIAL INFARCTION, DISCHARGED A...                        1   
               282 - ACUTE MYOCARDIAL INFARCTION, DISCHARGED A...                        1   
...                                                                                    ...   
WY             419 - LAPAROSCOPIC CHOLECYSTECTOMY W/O C.D.E. W...                        1   
               439 - DISORDERS OF PANCREAS EXCEPT MALIGNANCY W CC                        2   
               460 - SPINAL FUSION EXCEPT CERVICAL W/O MCC                               4   
               470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT O...                       11   
               473 - CERVICAL SPINAL FUSION W/O CC/MCC                                   3   
               480 - HIP & FEMUR PROCEDURES EXCEPT MAJOR JOINT...                        1   
               481 - HIP & FEMUR PROCEDURES EXCEPT MAJOR JOINT...                        3   
               482 - HIP & FEMUR PROCEDURES EXCEPT MAJOR JOINT...                        2   
               491 - BACK & NECK PROC EXC SPINAL FUSION W/O CC...                        3   
               536 - FRACTURES OF HIP & PELVIS W/O MCC                                   1   
               552 - MEDI